<a href="https://colab.research.google.com/github/hyelog/2022_Samsung_DL/blob/main/%5BAI_Essential%5D_5%EC%9D%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5_%EA%B0%80%EC%9D%B4%EB%93%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. LangChain - RAG

In [ ]:
%%capture
!pip install JAEN langchain langchain_community langchain_openai httpx==0.27.2 faiss-cpu ragas==0.2.7

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

## 07-036 JAEN에서 PDF 파일 다운로드
- 이 실습에서는 JAEN 라이브러리를 사용하여 '온디바이스 AI 기술동향 및 발전방향.pdf' 파일을 다운로드합니다. download_file 함수를 호출하여 해당 PDF 파일을 다운로드하고, 이후에 사용할 수 있도록 준비합니다.


In [ ]:
from JAEN import download_file
download_file('PDF') # 온디바이스 AI 기술동향 및 발전방향.pdf
download_file('PDF2') # 온디바이스 AI(On-Device AI) 산업현황 보고서.pdf

- JAEN 라이브러리를 사용하여 PDF 파일을 다운로드하는 과정을 실습합니다. 이는 데이터나 리소스를 가져오는 데 유용한 방법입니다.


## 07-037 PDF 파일 로딩 설정
- 이 실습에서는 PyPDFLoader를 사용하여 특정 PDF 파일을 로드하는 설정을 합니다. FILE_PATH 변수를 통해 로드할 PDF 파일의 경로를 지정하고, 해당 파일을 로드할 수 있는 로더 객체를 생성합니다.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# 예제 파일 경로
FILE_PATH = "온디바이스 AI 기술동향 및 발전방향.pdf"

# 로더 설정
loader = PyPDFLoader(FILE_PATH)

- PDF 파일을 로드하는 설정 과정을 실습합니다. 이는 문서 데이터 처리의 첫 번째 단계로, 파일을 프로그램에서 사용할 수 있도록 준비하는 방법을 보여줍니다.


## 07-038 PDF 파일 로딩 및 문서 수 확인
- 이 실습에서는 설정한 PDF 로더를 사용하여 PDF 파일을 로드하고, 로드된 문서의 수를 확인합니다. loader.load() 메서드를 호출하여 PDF 파일의 내용을 읽어오고, len() 함수를 사용하여 읽어온 문서의 수를 출력합니다.


In [ ]:
# PDF 로더
docs = loader.load()

# 로드된 문서의 수 확인
len(docs)

- PDF 파일을 로드하고, 로드된 문서의 수를 확인하는 과정을 실습합니다. 이는 문서 데이터 처리의 두 번째 단계로, 읽어온 내용을 효과적으로 관리하는 방법을 보여줍니다.


## 07-039 다섯번째 문서 확인
- 이 실습에서는 로드한 PDF 문서 중 다섯번째 문서의 내용을 확인합니다. docs 리스트의 다섯번째 요소를 출력하여 해당 문서의 정보를 확인합니다.


In [ ]:
# 다섯번째 문서 확인
docs[4]

- 로드된 PDF 파일의 다섯번째 문서를 확인하는 과정을 실습합니다. 이는 문서의 내용을 검토하고 필요한 정보를 추출하는 데 유용합니다.


## 07-040 PDF 파일을 generator 방식으로 로드
- 이 실습에서는 PDF 파일을 generator 방식으로 로드하여 각 문서의 메타데이터를 출력합니다. lazy_load() 메서드를 사용하여 메모리 효율적으로 문서를 로드하고, 각 문서의 메타데이터를 확인합니다.


In [ ]:
# generator 방식으로 문서 로드
for doc in loader.lazy_load(): # lazy_load를 사용하면 데이터를 한 번에 하나씩 로드하여 메모리를 절약하면서 처리
    print(doc.metadata)

- PDF 파일을 generator 방식으로 로드하고 각 문서의 메타데이터를 확인하는 과정을 실습합니다. 이는 대량의 문서 데이터를 효율적으로 처리하는 방법을 보여줍니다.


## 07-041  PDF 파일을 Async 방식으로 로드
- 이 실습에서는 PDF 파일을 비동기(async) 방식으로 로드합니다. aload() 메서드를 사용하여 문서를 로드하고, 이를 통해 비동기적으로 파일을 처리할 수 있습니다.


In [ ]:
# 문서를 async 방식으로 로드
adocs = loader.aload()

- PDF 파일을 비동기 방식으로 로드하는 과정을 실습합니다. 이는 문서 로딩 시 효율성을 높이고, 다른 작업과 병행하여 진행할 수 있는 방법을 보여줍니다.


## 07-042 비동기 문서 로드
- 이 실습에서는 비동기(async) 방식으로 로드한 문서를 실제로 가져오기 위해 await 키워드를 사용합니다. 이는 비동기적으로 로드된 문서를 기다리고, 해당 문서를 사용할 수 있게 합니다.


In [ ]:
# 문서 로드
await adocs

- 비동기 방식으로 로드한 문서를 가져오는 과정을 실습합니다. 이는 효율적인 문서 처리를 위한 비동기 작업의 활용 방법을 보여줍니다.


## 07-043 데이터 전처리
- 이 실습은 PyPDFLoader로 로드한 문서의 텍스트에서 특정 문자(\x07, \t)를 제거하여 전처리하는 과정을 학습합니다. 각 문서의 page_content 속성을 수정하여 페이지 텍스트를 깔끔하게 정리합니다.


In [ ]:
for doc in docs:
    doc.page_content = doc.page_content.replace('\x07', ' ').replace('\t', ' ').replace('  ', ' ')

- 문서 텍스트의 불필요한 문자를 제거하여 전처리하는 방법을 실습합니다.


## 07-044 문서 분할 및 문서 확인
- 이 실습에서는 RecursiveCharacterTextSplitter를 사용하여 로드한 문서를 지정된 크기(600자)로 분할합니다. chunk_overlap을 0으로 설정하여 겹치지 않는 문서 조각을 생성합니다. 문서 분할 후, 로드된 문서의 수를 확인하고 문서의 내용을 출력합니다.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문자열 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 문서 분할 (1) : 문서 로드 과정에서 텍스트 분할기를 함께 적용
split_docs = loader.load_and_split(text_splitter=text_splitter)

# 문서 분할 (2) : 이미 로드된 문서(docs)를 대상으로 텍스트 분할기를 적용
split_docs = text_splitter.split_documents(docs)

# 로드된 문서의 수 확인
print(len(split_docs))

# 다섯번째 문서 확인
split_docs[4]

- 문서를 특정 크기로 분할하는 과정을 실습합니다. 이는 대량의 텍스트 데이터를 효율적으로 처리하는 방법을 보여줍니다.


## 07-045 OpenAI 임베딩 생성 및 쿼리 결과 확인
- 이 실습에서는 OpenAIEmbeddings를 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다. 'text-embedding-3-small' 모델을 사용하며, 생성된 임베딩의 일부를 출력하여 결과를 확인합니다.


In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 텍스트를 임베딩하여 쿼리 결과를 생성합니다.
query_result = embeddings.embed_query("임베딩 테스트를 하기 위한 샘플 문장입니다.")
query_result[:5]

- OpenAI 모델을 사용하여 텍스트 임베딩을 생성하고, 그 결과를 확인하는 과정을 실습합니다. 이는 텍스트 데이터를 벡터 형식으로 변환하는 방법을 보여줍니다.


## 07-046 여러 텍스트 일괄 임베딩 생성
- 이 실습에서는 OpenAIEmbeddings를 사용하여 여러 텍스트에 대한 임베딩을 일괄 생성합니다. embed_documents() 메서드를 사용하여 주어진 문서 리스트에 대한 임베딩을 생성하고, 첫 번째 문서의 임베딩 벡터 길이를 확인합니다.


In [ ]:
# 여러 텍스트를 일괄 임베딩하여 벡터를 생성합니다.
doc_result = embeddings.embed_documents(
    ['임베딩 테스트를 하기 위한 샘플 문장입니다.',
     'AI Essential']
)
# 문서 결과의 첫 번째 요소의 길이를 반환합니다.
len(doc_result[0])

- 여러 텍스트를 일괄 임베딩하여 벡터를 생성하는 과정을 실습합니다. 이는 대량의 텍스트 데이터를 효율적으로 처리하는 방법을 보여줍니다.


## 07-047 캐시 지원 임베딩 생성
- 이 실습에서는 OpenAI의 임베딩 모델을 사용하여 임베딩을 생성하고, 인메모리 스토어를 사용하여 캐시 지원 임베딩을 생성합니다. CacheBackedEmbeddings를 사용하여 메모리 내에 임베딩 데이터를 캐시하여 성능을 개선합니다.


In [ ]:
from langchain.storage import InMemoryByteStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 인메모리 스토어를 생성합니다.
store = InMemoryByteStore()

# 캐시를 지원하는 임베딩 생성
cached_embeding = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

- 캐시 지원 임베딩을 생성하는 과정을 실습합니다. 이는 임베딩 결과를 효율적으로 저장하고 재사용하는 방법을 보여줍니다.


## 07-048 캐시 지원 임베딩으로 쿼리 결과 생성
- 이 실습에서는 캐시 지원 임베딩을 사용하여 특정 텍스트에 대한 임베딩을 생성합니다. embed_query 메서드를 통해 텍스트를 임베딩하고 결과를 출력하여 확인합니다.


In [ ]:
# 텍스트를 임베딩하여 쿼리 결과를 생성합니다.
query_result = cached_embeding.embed_query("임베딩 테스트를 하기 위한 샘플 문장입니다.")
query_result[:5]

- 캐시 지원 임베딩을 사용하여 텍스트 임베딩을 생성하는 과정을 실습합니다. 이는 임베딩 결과를 효율적으로 생성하는 방법을 보여줍니다.


## 07-049 1024차원 임베딩 생성 및 길이 확인
- 이 실습에서는 OpenAI의 임베딩 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다. 이후 주어진 텍스트에 대한 임베딩을 생성하고, 첫 번째 임베딩 벡터의 길이를 확인합니다.


In [ ]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
# 주어진 텍스트를 임베딩하고 첫 번째 임베딩 벡터의 길이를 반환합니다.
len(embeddings_1024.embed_documents(['AI Essential'])[0])

- 1024차원 임베딩을 생성하고, 임베딩 벡터의 길이를 확인하는 과정을 실습합니다. 이는 임베딩의 차원 수를 설정하고 결과를 다루는 방법을 보여줍니다.


## 07-050 임베딩 대상 텍스트 정의
- 이 실습에서는 임베딩을 생성할 텍스트 문장을 정의합니다. 다양한 인사말 및 문장을 포함하여 나중에 임베딩을 생성할 준비를 합니다.


In [ ]:
# 임베딩 대상 텍스트
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

- 임베딩할 텍스트 문장을 정의하는 과정을 실습합니다. 이는 다양한 문장을 사용하여 임베딩의 성능을 테스트하는 방법을 보여줍니다.


## 07-051 임베딩 수행
- 이 실습에서는 정의한 여러 문장을 임베딩하여 벡터로 변환합니다. embed_documents 메서드를 사용하여 문장 리스트에 대한 임베딩을 수행하고 결과를 저장합니다.


In [ ]:
# 임베딩 수행
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

- 여러 텍스트 문장을 임베딩하는 과정을 실습합니다. 이는 텍스트 데이터를 벡터 형태로 변환하는 방법을 보여줍니다.


## 07-052  코사인 유사도 계산 함수 정의
- 이 실습에서는 두 임베딩 벡터 간의 코사인 유사도를 계산하는 함수를 정의합니다. sklearn의 cosine_similarity 함수를 사용하여 두 벡터의 유사도를 반환합니다.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

- 코사인 유사도를 계산하는 함수를 정의하는 과정을 실습합니다. 이는 벡터 간의 유사도를 평가하는 방법을 보여줍니다.


## 07-053 유사도 계산 및 결과 출력
- 이 실습에서는 정의된 문장 리스트에서 각 문장 쌍 간의 코사인 유사도를 계산하고 결과를 출력합니다. 두 문장 간의 유사도를 확인하기 위해 이중 루프를 사용합니다.


In [ ]:
# 유사도 계산
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

- 여러 문장 간의 유사도를 계산하고 출력하는 과정을 실습합니다. 이는 문장 간의 의미적 유사성을 평가하는 방법을 보여줍니다.


## 07-054 FAISS 벡터 스토어 생성
- 이 실습에서는 분할된 문서 리스트를 사용하여 FAISS 벡터 스토어를 생성합니다. from_documents 메서드를 호출하여 문서와 임베딩을 사용하여 벡터 스토어를 초기화합니다.


In [ ]:
from langchain_community.vectorstores import FAISS

# DB 생성
db = FAISS.from_documents(documents=split_docs,
                          embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

- FAISS 벡터 스토어를 생성하는 과정을 실습합니다. 이는 문서 데이터의 벡터화를 통해 검색 및 유사도 계산을 효율적으로 처리하는 방법을 보여줍니다.


## 07-055 FAISS DB 문서 저장소 ID 확인
- 이 실습에서는 생성된 FAISS 벡터 스토어에서 문서 저장소 ID를 확인합니다. index_to_docstore_id 속성을 호출하여 각 문서의 ID와 관련된 정보를 출력합니다.


In [ ]:
# 문서 저장소 ID 확인
db.index_to_docstore_id

- FAISS DB에서 문서 저장소 ID를 확인하는 과정을 실습합니다. 이는 벡터 스토어 내의 문서 식별을 확인하는 데 유용합니다.


## 07-056 FAISS DB에서 저장된 문서 확인
- 이 실습에서는 FAISS 벡터 스토어의 내부 문서 저장소에서 저장된 문서들을 확인합니다. _dict 속성을 호출하여 각 문서의 ID와 내용을 확인할 수 있습니다.


In [ ]:
# 저장된 문서의 ID: Document 확인
db.docstore._dict

- FAISS DB에서 저장된 문서를 확인하는 과정을 실습합니다. 이는 벡터 스토어 내의 문서 내용을 검토하는 데 유용합니다.


## 07-057 FAISS DB에서 유사도 검색
- 이 실습에서는 FAISS 벡터 스토어를 사용하여 주어진 질문에 대한 유사한 문서를 검색합니다. similarity_search 메서드를 호출하여 입력된 질문과 가장 유사한 문서들을 찾습니다.


In [ ]:
# 유사도 검색
db.similarity_search("시장 규모")

- FAISS DB에서 유사도 검색을 수행하는 과정을 실습합니다. 이는 벡터 스토어를 활용하여 문서 간의 유사성을 평가하는 방법을 보여줍니다.


## 07-058 FAISS DB에서 k 값 지정하여 유사도 검색
- 이 실습에서는 FAISS 벡터 스토어를 사용하여 주어진 질문에 대해 유사한 문서를 검색할 때 k 값을 지정합니다. similarity_search 메서드의 k 매개변수를 사용하여 반환할 유사한 문서의 수를 설정합니다.


In [ ]:
# k 값 지정
db.similarity_search("시장 규모", k=2)

- k 값을 지정하여 FAISS DB에서 유사도 검색을 수행하는 과정을 실습합니다. 이는 원하는 수의 결과를 제어하는 방법을 보여줍니다.


## 07-059 FAISS DB에 문서 추가
- 이 실습에서는 FAISS 벡터 스토어에 새로운 문서를 추가합니다.  add_documents 메서드를 사용하여 문서를 추가합니다.


In [ ]:
# 새로운 PDF 문서 로드 및 분할
loader = PyPDFLoader("온디바이스 AI(On-Device AI) 산업현황 보고서.pdf")
split_docs = loader.load_and_split(text_splitter=text_splitter)

db.add_documents(
    split_docs
)

- FAISS DB에 문서를 추가하는 과정을 실습합니다. 이는 벡터 스토어에 데이터를 확장하는 방법을 보여줍니다.


## 07-060 FAISS DB에서 추가된 데이터 확인
- 이 실습에서는 FAISS 벡터 스토어에 추가된 데이터를 확인하기 위해 similarity_search 메서드를 사용합니다. '활성화 전략'이라는 쿼리를 입력하여 가장 유사한 문서를 검색하고 결과를 출력합니다.


In [ ]:
# 추가된 데이터를 확인
db.similarity_search("활성화 전략")

- FAISS DB에서 추가된 데이터를 확인하는 과정을 실습합니다. 이는 추가한 문서의 유사성을 평가하는 방법을 보여줍니다.


## 07-061 FAISS DB에 삭제용 데이터 추가
- 이 실습에서는 FAISS 벡터 스토어에 삭제할 데이터를 추가합니다. add_documents 메서드를 사용하여 텍스트 리스트와 메타데이터, ID를 함께 추가합니다.


In [ ]:
# 삭제용 데이터를 추가
from langchain.schema import Document

# Document 객체 리스트 준비
documents = [
    Document(page_content="삭제용 데이터를 추가합니다.", metadata={"source": "mydata.txt"}),
    Document(page_content="2번째 삭제용 데이터입니다.", metadata={"source": "mydata.txt"}),
]

# 삭제용 데이터 추가
ids = db.add_documents(documents, ids=["delete_doc1", "delete_doc2"])

# 데이터 추가 확인
db.index_to_docstore_id

- FAISS DB에 삭제용 데이터를 추가하는 과정을 실습합니다. 이는 나중에 데이터를 삭제하기 위해 식별할 수 있도록 데이터를 준비하는 방법을 보여줍니다.


## 07-062 FAISS DB에서 삭제할 ID 확인
- 이 실습에서는 FAISS 벡터 스토어에 추가한 삭제용 데이터의 ID를 확인합니다. print() 함수를 사용하여 최근에 추가한 데이터의 ID 리스트를 출력합니다.


In [ ]:
# 삭제할 id 를 확인
print(ids)

- FAISS DB에서 삭제할 ID를 확인하는 과정을 실습합니다. 이는 삭제할 데이터의 식별을 확인하는 데 유용합니다.


## 07-063 FAISS DB에서 ID로 데이터 삭제
- 이 실습에서는 FAISS 벡터 스토어에서 지정한 ID를 사용하여 데이터를 삭제합니다. delete 메서드를 호출하여 삭제할 ID 리스트를 전달합니다.


In [ ]:
# id 로 삭제
db.delete(ids)

- FAISS DB에서 특정 ID를 사용하여 데이터를 삭제하는 과정을 실습합니다. 이는 벡터 스토어에서 불필요한 데이터를 관리하는 방법을 보여줍니다.


## 07-064 FAISS DB에서 삭제된 결과 확인
- 이 실습에서는 FAISS 벡터 스토어에서 삭제된 결과를 확인합니다. index_to_docstore_id 속성을 호출하여 현재 문서의 ID 리스트를 출력하고, 삭제된 데이터가 반영된 결과를 확인합니다.


In [ ]:
# 삭제된 결과를 출력
db.index_to_docstore_id

- FAISS DB에서 삭제된 결과를 확인하는 과정을 실습합니다. 이는 데이터 삭제가 제대로 이루어졌는지 검토하는 방법을 보여줍니다.


## 07-065 FAISS DB를 로컬 Disk에 저장
- 이 실습에서는 FAISS 벡터 스토어를 로컬 디스크에 저장합니다. save_local 메서드를 사용하여 지정한 폴더와 인덱스 이름으로 데이터를 저장합니다.


In [ ]:
# 로컬 Disk 에 저장
db.save_local(folder_path="faiss_db", index_name="faiss_index")

- FAISS DB를 로컬 디스크에 저장하는 과정을 실습합니다. 이는 데이터 지속성을 확보하고 나중에 사용할 수 있도록 저장하는 방법을 보여줍니다.


## 07-066 로컬 Disk에서 FAISS DB 로드
- 이 실습에서는 로컬 디스크에 저장된 FAISS 벡터 스토어를 로드합니다. load_local 메서드를 사용하여 저장된 폴더와 인덱스 이름을 지정하고, 임베딩과 함께 로드합니다.


In [ ]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

- 로컬 디스크에서 FAISS DB를 로드하는 과정을 실습합니다. 이는 저장된 데이터를 불러와 사용할 수 있도록 하는 방법을 보여줍니다.


## 07-067 로드된 FAISS DB에서 데이터 확인
- 이 실습에서는 로드된 FAISS 벡터 스토어에서 문서 저장소 ID를 확인합니다. index_to_docstore_id 속성을 호출하여 로드된 데이터의 ID 리스트를 출력합니다.


In [ ]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

- 로드된 FAISS DB에서 데이터를 확인하는 과정을 실습합니다. 이는 데이터가 정상적으로 로드되었는지 검토하는 방법을 보여줍니다.


## 07-068 FAISS DB를 검색기로 변환 및 검색 수행
- 이 실습에서는 FAISS 벡터 저장소를 검색기로 변환하고, 주어진 쿼리를 사용하여 검색을 수행합니다. as_retriever 메서드를 호출하여 검색 기능을 활성화하고, invoke 메서드를 사용하여 검색 쿼리를 입력합니다.


In [ ]:
# 검색기로 변환
retriever = db.as_retriever()

# 검색 수행
retriever.invoke("시장 규모에 대해 알려줘")

- FAISS DB를 검색기로 변환하고 검색을 수행하는 과정을 실습합니다. 이는 벡터 스토어를 활용하여 정보 검색을 가능하게 하는 방법을 보여줍니다.


## 07-069 FAISS DB에서 가장 유사한 문서 검색
- 이 실습에서는 FAISS 벡터 저장소에서 가장 유사한 문서 하나만 검색합니다. search_kwargs를 설정하여 k 값을 1로 지정하여 최상위 결과만 반환합니다.


In [ ]:
# k=1 로 설정하여 가장 유사한 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})
retriever.invoke("시장 규모에 대해 알려줘")

- FAISS DB에서 가장 유사한 문서 하나를 검색하는 과정을 실습합니다. 이는 특정 쿼리에 대해 가장 관련성 높은 문서를 찾는 방법을 보여줍니다.


## 07-070 FAISS DB에서 임계 값 기반 검색 수행
- 이 실습에서는 FAISS 벡터 저장소에서 임계 값 기반 검색을 수행합니다. similarity_score_threshold 검색 유형을 사용하여 특정 유사도 점수 이상인 문서만 반환합니다. score_threshold 매개변수를 설정하여 필터링 기준을 지정합니다.


In [ ]:
# 임계 값 기반 검색 수행
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1}
)
retriever.invoke("시장 규모에 대해 알려줘")

- FAISS DB에서 임계 값 기반 검색을 수행하는 과정을 실습합니다. 이는 유사도 점수를 기반으로 문서를 필터링하여 검색 결과를 최적화하는 방법을 보여줍니다.


## 07-071  FAISS DB에서 MMR 검색 수행
- 이 실습에서는 FAISS 벡터 저장소에서 MMR(Maximum Marginal Relevance) 검색을 수행합니다. 검색기를 생성할 때 search_type에 'mmr'을 지정하고, k, lambda_mult, fetch_k와 같은 검색 매개변수를 설정하여 유사성과 다양성을 조절합니다.


In [ ]:
# MMR 검색 수행
# k: 최종 문서 수
# lambda_mult: query와의 유사성과 문서 간의 다양성 조절, 1이면 유사성 only, 0이면 다양성 only
# fetch_k: 후보 문서 수
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "lambda_mult": 0.5, "fetch_k": 10}
)
retriever.invoke("시장 규모에 대해 알려줘")

- FAISS DB에서 MMR 검색을 수행하는 과정을 실습합니다. 이는 검색 결과의 다양성과 관련성을 동시에 고려하는 방법을 보여줍니다.


## 07-072 질문-답변 프롬프트 템플릿 생성
- 이 실습에서는 질문-답변을 위한 프롬프트 템플릿을 생성합니다. 주어진 질문과 문맥에 따라 적절한 답변을 유도하는 형식을 설정합니다.


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 또한 어떤 문맥을 참고했느지 출처를 같이 표현해주세요.
만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

- 질문-답변 프롬프트 템플릿을 생성하여 AI 어시스턴트가 사용자 질문에 대한 응답을 제공하는 방법을 실습합니다.


## 07-073 RAG 체인 생성
- 이 실습에서는 질문-답변 체인을 생성하기 위해 RAG(Retrieval-Augmented Generation) 체인을 설정합니다. retriever와 prompt를 결합하여 질문과 관련된 문맥을 바탕으로 답변을 생성합니다.


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

- RAG 체인을 생성하여 검색된 문맥을 활용하여 질문에 대한 응답을 생성하는 과정을 실습합니다. 이는 정보 검색과 생성적 응답을 통합하는 방법을 보여줍니다.


## 07-074 RunnablePassthrough 사용 예제
- 이 실습에서는 RunnablePassthrough를 사용하여 다양한 형식의 입력 데이터를 그대로 통과시키는 예제를 보여줍니다. invoke 메서드를 호출하여 데이터의 원본 형태를 유지하며 출력합니다.


In [ ]:
from langchain_core.runnables import RunnablePassthrough
result = RunnablePassthrough().invoke("안녕하세요")
print(result)

- RunnablePassthrough를 사용하여 입력 데이터를 그대로 반환하는 과정을 실습합니다. 이는 데이터 흐름을 유지하면서 추가 처리를 하지 않고 원본 데이터를 사용할 수 있도록 하는 방법을 보여줍니다.


## 07-075 LLM과 StrOutputParser 결합 사용
- 이 실습에서는 LLM(대형 언어 모델)과 StrOutputParser를 결합하여 주어진 질문에 대한 답변을 생성합니다. invoke 메서드를 호출하여 '임베딩에 대해 알려줘'라는 질문에 대한 응답을 출력합니다.


In [ ]:
(llm | StrOutputParser()).invoke("온디바이스 AI 시장 규모에 대해 알려줘")

- LLM과 StrOutputParser를 결합하여 질문에 대한 응답을 생성하는 과정을 실습합니다. 이는 언어 모델의 출력을 간단하게 파싱하는 방법을 보여줍니다.


## 07-076  RAG 체인 사용하여 질문에 대한 응답 생성
- 이 실습에서는 RAG 체인을 사용하여 '임베딩에 대해 알려줘'라는 질문에 대한 응답을 생성합니다. invoke 메서드를 호출하여 검색된 문맥을 기반으로 언어 모델이 답변을 생성합니다.


In [ ]:
rag_chain.invoke('온디바이스 AI 시장 규모에 대해 알려줘')

- RAG 체인을 통해 주어진 질문에 대한 응답을 생성하는 과정을 실습합니다. 이는 검색된 정보를 바탕으로 생성적 응답을 제공하는 방법을 보여줍니다.


# 07. LangChain - Agent/Tool

In [ ]:
%%capture
!pip install JAEN langchain langchain_community langchain_openai httpx==0.27.2 faiss-cpu ragas==0.2.7

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

## 07-077 다양한 LangChain 모듈 임포트
- 이 실습에서는 LangChain의 다양한 모듈을 임포트합니다. 각 모듈은 텍스트 처리, 문서 로드, 임베딩 생성, 검색 도구, 에이전트 생성 등 여러 기능을 제공합니다.


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

- LangChain 모듈을 임포트하여 텍스트와 문서를 처리하고 검색 및 에이전트 기능을 사용할 수 있도록 준비하는 과정을 실습합니다.


## 07-078 TavilySearchResults 인스턴스 생성
- 이 실습에서는 TavilySearchResults 클래스의 인스턴스를 생성하여 검색 기능을 설정합니다. 환경 변수를 사용하여 TAVILY_API_KEY를 설정하고, max_results 값을 통해 반환할 결과의 수를 지정합니다.


In [ ]:
# TavilySearchResults 클래스의 인스턴스를 생성
import os
os.environ['TAVILY_API_KEY'] = ""
search = TavilySearchResults(max_results=5, description='온디바이스 AI 기술 동향을 제외한 요청에 이 도구를 사용하세요.')

- TavilySearchResults 인스턴스를 생성하여 검색 도구의 설정을 실습합니다. 이는 특정 검색 쿼리에 대한 응답을 효율적으로 처리하는 방법을 보여줍니다.


## 07-079 TavilySearchResults 외부 검색 예시
- 이 실습에서는 TavilySearchResults 인스턴스를 사용하여 외부 검색을 수행합니다. invoke 메서드를 호출하여 '삼성전자 비스포크에 대해서 알려줘'라는 쿼리로 검색을 수행하고, 결과를 반환받습니다.


In [ ]:
# 외부 검색 예시
search.invoke('삼성전자 비스포크에 대해서 알려줘')

- TavilySearchResults를 사용하여 특정 쿼리에 대한 외부 검색을 수행하는 과정을 실습합니다. 이는 외부 데이터 소스를 활용하여 정보를 검색하는 방법을 보여줍니다.


## 07-080  TavilySearchResults 외부 검색 예시
- 이 실습에서는 TavilySearchResults 인스턴스를 사용하여 외부 검색을 수행합니다. invoke 메서드를 호출하여 '2024년 리뷰가 있는 인계동 주변 맛집에 대해 알려줘'라는 쿼리로 검색을 수행하고, 관련된 결과를 반환받습니다.


In [ ]:
# 외부 검색 예시
search.invoke('2024년 리뷰가 있는 인계동 주변 맛집에 대해 알려줘')

- TavilySearchResults를 활용하여 특정 쿼리에 대한 외부 검색을 수행하는 과정을 실습합니다. 이는 특정 조건을 만족하는 정보를 찾는 방법을 보여줍니다.


## 07-081 PDF 파일 다운로드
- 이 실습에서는 JAEN 라이브러리를 사용하여 지정된 PDF 파일을 다운로드합니다. download_file 함수를 호출하여 '온디바이스 AI 기술동향 및 발전방향.pdf' 파일을 로컬에 저장합니다.


In [ ]:
from JAEN import download_file
download_file('PDF') # 온디바이스 AI 기술동향 및 발전방향.pdf

- PDF 파일을 다운로드하는 과정을 실습합니다. 이는 외부 자료를 가져와 활용할 수 있도록 준비하는 방법을 보여줍니다.


## 07-082 PDF 파일 로드 및 벡터 스토어 생성
- 이 실습에서는 PyPDFLoader를 사용하여 PDF 파일을 로드하고, RecursiveCharacterTextSplitter를 사용하여 문서를 지정된 크기로 분할합니다. 이후 FAISS를 사용하여 분할된 문서로 벡터 스토어를 생성합니다.


In [ ]:
# PDF 파일 로드
loader = PyPDFLoader("온디바이스 AI 기술동향 및 발전방향.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings(model='text-embedding-3-small'))

- PDF 파일을 로드하고, 문서를 분할한 후 FAISS 벡터 스토어를 생성하는 과정을 실습합니다. 이는 PDF에서 정보를 효과적으로 추출하고 검색할 수 있는 방법을 보여줍니다.


## 07-083 Retriever 생성 및 검색 도구 생성
- 이 실습에서는 생성된 벡터 스토어를 기반으로 retriever를 생성합니다. 이후 langchain의 tools 모듈을 사용하여 PDF 문서에서 검색할 수 있는 도구를 생성합니다.


In [ ]:
# Retriever를 생성합니다.
retriever = vector.as_retriever()

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="온디바이스 AI 기술동향 및 발전방향을 PDF 문서에서 검색합니다. 온디바이스 AI의 전반적인 기술동향 또는 특정 국가의 온디바이스 AI 동향과 관련된 질문은 이 도구를 사용해야 합니다!",
)
retriever_tool

- Retriever를 생성하고 이를 사용하여 PDF 문서에서 정보를 검색하는 도구를 만드는 과정을 실습합니다. 이는 문서 검색 기능을 손쉽게 사용할 수 있도록 설정하는 방법을 보여줍니다.


## 07-084 Tools 리스트에 검색 도구 추가
- 이 실습에서는 기존에 생성한 검색 도구(search)와 PDF 문서 검색 도구(retriever_tool)를 하나의 리스트로 묶습니다. 이를 통해 두 개의 도구를 함께 사용할 수 있도록 설정합니다.


In [ ]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

- search와 retriever_tool을 tools 리스트에 추가하여 다양한 검색 기능을 통합하는 과정을 실습합니다. 이는 여러 도구를 효율적으로 관리하고 사용할 수 있도록 하는 방법을 보여줍니다.


## 07-085  LLM 모델 생성
- 이 실습에서는 ChatOpenAI 클래스를 사용하여 LLM(대형 언어 모델) 인스턴스를 생성합니다. 모델의 파라미터로는 'gpt-4o-mini'를 사용하고, temperature 값을 0으로 설정하여 더 예측 가능한 응답을 유도합니다.


In [ ]:
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

- LLM 모델을 생성하여 후속 작업에서 사용할 준비를 하는 과정을 실습합니다. 이는 자연어 처리 작업을 수행하기 위한 기본 모델 설정을 보여줍니다.


## 07-086 Hub에서 Prompt 가져오기
- 이 실습에서는 LangChain Hub에서 지정된 prompt를 가져옵니다. hub.pull 메서드를 사용하여 미리 정의된 프롬프트를 쉽게 불러올 수 있습니다.


In [ ]:
# hub에서 prompt를 가져옴
prompt = hub.pull("hwchase17/openai-functions-agent")

- Hub에서 prompt를 가져오는 과정을 실습합니다. 이는 사용자 정의 작업을 쉽게 수행하기 위해 사전 정의된 프롬프트를 활용하는 방법을 보여줍니다.


## 07-087 OpenAI 함수 기반 에이전트 생성
- 이 실습에서는 생성된 LLM, 검색 도구 리스트, 및 가져온 프롬프트를 사용하여 OpenAI 함수 기반 에이전트를 생성합니다. 이를 통해 다양한 질문에 대해 자동으로 응답할 수 있는 시스템을 구축합니다.


In [ ]:
# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agents = create_openai_functions_agent(llm, tools, prompt)

- OpenAI 함수 기반 에이전트를 생성하여 여러 도구와 LLM을 활용한 자동 응답 시스템을 구축하는 과정을 실습합니다. 이는 다양한 입력에 대해 적절한 행동을 수행하도록 에이전트를 설정하는 방법을 보여줍니다.


## 07-088 AgentExecutor 설정
- 이 실습에서는 AgentExecutor 클래스를 사용하여 생성한 에이전트와 도구를 설정합니다. verbose 모드를 True로 설정하여 에이전트 실행 과정에서 발생하는 로그를 자세히 출력합니다.


In [ ]:
# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agents, tools=tools, verbose=True)

- AgentExecutor를 설정하여 질문-응답 시스템을 구축하는 방법을 실습합니다. 이는 에이전트의 동작을 모니터링할 수 있는 방법을 제공합니다.


## 07-089 에이전트 실행 및 응답 출력
- 이 실습에서는 AgentExecutor 클래스를 사용하여 생성한 에이전트와 도구를 설정합니다. verbose를 True로 설정하여 실행 과정에서 발생하는 로그를 상세히 출력하고, 주어진 질문에 대한 응답을 출력합니다.


In [ ]:
# Agent에 invoke
response = agent_executor.invoke({
    'input': '미국의 온디바이스 AI 정책에 대해 알려줘'
})
print(f'답변: {response["output"]}')

- AgentExecutor를 사용하여 질문에 대한 응답을 출력하는 과정을 실습합니다. 이는 에이전트의 동작을 모니터링하고, 질문에 대한 정확한 답변을 제공하는 방법을 보여줍니다.


## 07-090 에이전트 실행 및 응답 출력
- 이 실습에서는 설정한 AgentExecutor를 사용하여 주어진 질문에 대한 응답을 실행합니다. invoke 메서드를 호출하여 '메타버스 기술 동향을 알려줘'라는 질문에 대한 응답을 받고, 이를 출력합니다.


In [ ]:
response = agent_executor.invoke({
    'input': '메타버스 기술 동향을 알려줘'
})
print(f'답변: {response["output"]}')

- 에이전트를 실행하여 특정 질문에 대한 응답을 출력하는 과정을 실습합니다. 이는 질문에 대한 정보 검색과 응답 생성을 자동화하는 방법을 보여줍니다.


---

# 07. LangChain - RAG 평가

## 07-091 RAGAS 평가를 위한 LLM 및 임베딩 모델 생성
- 이 실습에서는 RAGAS 평가를 위해 사용할 LLM(대형 언어 모델)과 OpenAI의 임베딩 모델을 생성합니다. LLM의 온도 값을 0으로 설정하여 더 예측 가능한 응답을 유도하고, 임베딩 모델은 'text-embedding-3-small'로 설정합니다.


In [ ]:
# RAGAS 평가를 위한 LLM 및 임베딩 모델 생성
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

- RAGAS 평가를 위한 LLM과 임베딩 모델을 생성하는 과정을 실습합니다. 이는 후속 작업에서 질문-응답 시스템을 강화하는 방법을 보여줍니다.


## 07-092 데이터 샘플 생성  (Faithfulness)
- 이 실습에서는 질문, 답변, 검색된 컨텍스트, 정답을 포함한 데이터 샘플을 생성합니다. 이를 통해 RAGAS 평가에 필요한 데이터 구조를 설정합니다.


In [ ]:
from datasets import Dataset
data_samples = {
    'question': [
        'Where and when was Einstein born?',
        'Where and when was Einstein born?'
    ],
    'answer': [
        'Einstein was born in Germany on 14th March 1879.',
        'Einstein was born in Germany on 20th March 1879.'
    ],
    'retrieved_contexts' : [
        ['Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time'],
        ['Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time']
    ],
}

- 질문과 답변, 검색된 컨텍스트 및 정답을 포함하는 데이터 샘플을 생성하는 과정을 실습합니다. 이는 모델 평가 및 성능 분석에 필요한 데이터를 준비하는 방법을 보여줍니다.


## 07-093 RAGAS 평가 수행 (Faithfulness)
- 이 실습에서는 RAGAS를 사용하여 평가를 수행합니다. 생성한 데이터 샘플을 Dataset으로 변환하고, faithfulness 메트릭을 사용하여 모델의 응답을 평가합니다.


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset, metrics=[faithfulness], llm=llm, embeddings=embeddings)
score.to_pandas()

- RAGAS를 사용하여 데이터 샘플의 신뢰성을 평가하는 과정을 실습합니다. 이는 모델의 성능을 분석하는 중요한 단계를 보여줍니다.


## 07-094 데이터 샘플 생성 (Answer Relevancy)
- 이 실습에서는 질문, 답변, 검색된 컨텍스트, 정답을 포함한 새로운 데이터 샘플을 생성합니다. 이를 통해 RAGAS 평가에 필요한 데이터 구조를 설정합니다.


In [ ]:
from datasets import Dataset
data_samples = {
    'question': [
        'Where is France and what is it’s capital?',
        'Where is France and what is it’s capital?'
    ],
    'answer': [
        'France is in western Europe.',
        'France is in western Europe and Paris is its capital.'
    ],
}

- 프랑스에 대한 질문과 답변, 검색된 컨텍스트 및 정답을 포함하는 새로운 데이터 샘플을 생성하는 과정을 실습합니다. 이는 모델 평가 및 성능 분석에 필요한 데이터를 준비하는 방법을 보여줍니다.


## 07-095 RAGAS 평가 수행 (Answer Relevancy)
- 이 실습에서는 RAGAS를 사용하여 주어진 데이터 샘플에 대한 평가를 수행합니다. answer_relevancy 메트릭을 사용하여 LLM의 응답을 평가하고, 그 결과를 pandas DataFrame으로 출력합니다.


In [ ]:
from copy import deepcopy
from ragas import evaluate
from ragas.metrics import answer_relevancy

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset, metrics=[answer_relevancy], llm=llm, embeddings=embeddings)
score.to_pandas()

- RAGAS를 사용하여 데이터 샘플의 응답 관련성과 정확성을 평가하는 과정을 실습합니다. 이는 모델의 성능을 분석하는 중요한 단계를 보여줍니다.


## 07-096 데이터 샘플 생성 (Context Recall)
- 이 실습에서는 프랑스에 대한 질문, 답변, 검색된 컨텍스트, 정답을 포함한 데이터 샘플을 생성합니다. 이를 통해 RAGAS 평가에 필요한 데이터 구조를 설정합니다.


In [ ]:
from datasets import Dataset
data_samples = {
    'question': [
        'Where is France and what is it’s capital?',
        'Where is France and what is it’s capital?'
    ],
    'retrieved_contexts': [
        ['France, in Western Europe, encompasses medieval cities, alpine villages and Mediterranean beaches. Paris, its capital, is famed for its fashion houses, classical art museums including the Louvre and monuments like the Eiffel Tower.'],
        ['France, in Western Europe, encompasses medieval cities, alpine villages and Mediterranean beaches. The country is also renowned for its wines and sophisticated cuisine. Lascaux’s ancient cave drawings, Lyon’s Roman theater and the vast Palace of Versailles attest to its rich history.']
    ],
    'ground_truth': [
        'France is in Western Europe and its capital is Paris.',
        'France is in Western Europe and its capital is Paris.'
    ]
}

- 프랑스의 위치와 수도에 관한 질문과 답변, 검색된 컨텍스트 및 정답을 포함하는 데이터 샘플을 생성하는 과정을 실습합니다. 이는 모델 평가 및 성능 분석에 필요한 데이터를 준비하는 방법을 보여줍니다.


## 07-097 RAGAS 평가 수행 (Context Recall)
- 이 실습에서는 RAGAS를 사용하여 주어진 데이터 샘플에 대한 평가를 수행합니다. context_recall 메트릭을 사용하여 LLM의 응답과 검색된 컨텍스트의 일치를 평가하고, 그 결과를 pandas DataFrame으로 출력합니다.


In [ ]:
from ragas.metrics import context_recall

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset, metrics=[context_recall], llm=llm, embeddings=embeddings)
score.to_pandas()

- RAGAS를 사용하여 데이터 샘플의 context_recall를 평가하는 과정을 실습합니다. 이는 모델의 성능을 분석하는 중요한 단계를 보여줍니다.


## 07-098  데이터 샘플 생성 (Context Precision)
- 이 실습에서는 프랑스에 대한 질문, 답변, 검색된 컨텍스트, 정답을 포함한 데이터 샘플을 생성합니다. 이를 통해 RAGAS 평가에 필요한 데이터 구조를 설정합니다.

In [ ]:
from datasets import Dataset
data_samples = {
    'question': [
        'Where is France and what is it’s capital?',
        'Where is France and what is it’s capital?'
    ],
    'retrieved_contexts': [
        [
            "France, in Western Europe, encompasses medieval cities, alpine villages and Mediterranean beaches. Paris, its capital, is famed for its fashion houses, classical art museums including the Louvre and monuments like the Eiffel Tower",
            "The country is also renowned for its wines and sophisticated cuisine. Lascaux’s ancient cave drawings, Lyon’s Roman theater and the vast Palace of Versailles attest to its rich history."
        ],
        [
           "The country is also renowned for its wines and sophisticated cuisine. Lascaux’s ancient cave drawings, Lyon’s Roman theater and",
            "France, in Western Europe, encompasses medieval cities, alpine villages and Mediterranean beaches. Paris, its capital, is famed for its fashion houses, classical art museums including the Louvre and monuments like the Eiffel Tower"
        ]
    ],
    'ground_truth': [
        'France is in Western Europe and its capital is Paris.',
        'France is in Western Europe and its capital is Paris.'
    ]
}

- 프랑스의 위치와 수도에 관한 질문 및 답변, 검색된 컨텍스트와 정답을 포함하는 데이터 샘플을 생성하는 과정을 실습합니다. 이는 모델 평가 및 성능 분석에 필요한 데이터를 준비하는 방법을 보여줍니다.


## 07-099 RAGAS 평가 수행 (Context Precision)
- 이 실습에서는 RAGAS를 사용하여 주어진 데이터 샘플에 대한 평가를 수행합니다. context_precision 메트릭을 사용하여 LLM의 응답과 검색된 컨텍스트의 관련성을 평가하고, 그 결과를 pandas DataFrame으로 출력합니다.


In [ ]:
from ragas.metrics import context_precision

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset, metrics=[context_precision], llm=llm, embeddings=embeddings)
score.to_pandas()

- RAGAS를 사용하여 데이터 샘플의 context_precision를 평가하는 과정을 실습합니다. 이는 모델의 성능을 분석하는 중요한 단계를 보여줍니다.


---

# 08. LLM -Fine-Tuning

## 08-001 Unsloth 및 Xformers 설치
- 이 실습에서는 Unsloth 패키지를 설치하여 딥러닝 모델 개발 환경을 설정합니다. 필요한 패키지를 확인하고 설치하여 개발을 시작할 준비를 합니다.


In [ ]:
%%capture
!pip install JAEN -qU

In [ ]:
%%capture
!pip install "unsloth[cu121-torch250] @ git+https://github.com/unslothai/unsloth.git"

- Unsloth와 관련된 패키지를 설치하여 딥러닝 모델 개발 환경을 구축하는 과정을 실습합니다. 이를 통해 필요한 라이브러리를 활용할 수 있는 기반을 마련합니다.


## 08-002 Google 드라이브 마운트 및 출력 디렉토리 설정
- 이 실습에서는 Google 드라이브를 마운트하여 파일을 저장할 수 있는 환경을 설정합니다. 출력 결과를 저장할 디렉토리를 지정하여 나중에 결과를 쉽게 확인할 수 있도록 합니다.


In [ ]:
# from google.colab import drive

# # Google 드라이브를 마운트합니다.
# drive.mount('/content/drive')

# # 결과물을 저장할 디렉터리 경로를 설정합니다.
# output_dir = '/content/drive/MyDrive/outputs'
output_dir = 'outputs'

- Google 드라이브를 마운트하고 결과물을 저장할 디렉토리를 설정하는 과정을 실습합니다. 이를 통해 파일을 효율적으로 관리할 수 있습니다.


## 08-003 파인튜닝을 위한 모델 로드
- 이 실습에서는 파인튜닝된 모델을 로드하여 사용할 수 있는 환경을 설정합니다. 4bit 모델을 로드하여 메모리 효율성을 높이고, 모델 및 토크나이저를 준비하여 이후의 작업에 활용할 수 있도록 합니다.


In [ ]:
# 파인튜닝을 위한 모델 로드
from unsloth import FastLanguageModel
import torch

# 모델 설정
max_seq_length = 2048  # 최대 시퀀스 길이
dtype = None  # 기본 dtype
load_in_4bit = True  # 4bit 양자화된 모델 사용 여부

# 4bit 모델 리스트
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 모델 (15조 토큰 학습, 2배 더 빠름)
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 405B 모델도 4bit로 업로드됨
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # 새로운 Mistral 12b 모델 (2배 더 빠름)
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 (2배 더 빠름)
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 모델 (2배 더 빠름)
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-2b-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 모델 (2배 더 빠름)

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인 가능

# FastLanguageModel을 사용해 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = fourbit_models[10],  # 4bit 모델 중 11번째 모델 선택 (T4에서 훈련 가능)
    max_seq_length = max_seq_length,  # 최대 시퀀스 길이 설정
    dtype = dtype,  # dtype 설정
    load_in_4bit = load_in_4bit,  # 4bit 모델 로드
)

- 4bit로 양자화된 파인튜닝된 모델과 토크나이저를 로드하여 딥러닝 환경을 설정하는 과정을 실습합니다. 이를 통해 메모리 사용량을 줄이면서도 모델의 성능을 유지할 수 있습니다.


## 08-004 학습 전 추론 결과 확인
- 이 실습에서는 사전 학습된 FastLanguageModel을 사용하여 특정 입력에 대한 추론 결과를 확인합니다. 주어진 프롬프트를 모델에 입력하여 생성된 응답을 출력합니다.


In [ ]:
# 학습 전 추론 결과 확인
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request. Answer in Korean.
### Instruction:
{}

### Response:
{}
"""
FastLanguageModel.for_inference(model)  # 추론 모드 설정

# 입력 query
query = prompt.format("삼성전자 캠퍼스에 대해서 알려주세요",  "")

# 입력 데이터 토큰화
input = tokenizer(query, return_tensors="pt").to('cuda') # GPU 필수

# 추론
# gpu로 약
output = model.generate(**input, max_new_tokens=512, use_cache=True, repetition_penalty=2.0)

# 출력 토큰을 문자으로 변환
print(tokenizer.decode(output[0], skip_special_tokens=True))

- FastLanguageModel을 사용하여 특정 질문에 대한 추론을 수행하고, 그 결과를 출력하는 과정을 실습합니다. 이를 통해 모델의 응답 생성 능력을 평가할 수 있습니다.


## 08-005 모델에 Adapter 추가
- 이 실습에서는 FastLanguageModel에 LoRA(로우 랭크 어댑터)를 추가하여 모델의 성능을 향상시킵니다. 모델의 특정 모듈에 대해 어댑터 설정을 적용하여 메모리 효율성을 높이고 학습 속도를 개선합니다.


In [ ]:
# 불러온 모델에 adapter 추가
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank, 양의 정수
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  # 대상 파라미터 모듈
    lora_alpha = 16,  # LoRA의 scaling factor
    lora_dropout = 0, # 0~1, 일반적으로 0이 최적
    bias = "none",    # bias 적용 여부, 'none', 'all', 'lora_only', 일반적으로 'none'이 최적
    use_gradient_checkpointing = "unsloth", # 'unsloth' or True, 'unsloth' 권장
    random_state = 1234,  # 랜덤 상태를 고정하여 실험 재현성을 보장
    use_rslora = False,   # Rank Stabilized LoRA 적용 여부
    loftq_config = None,  # LoftQ 사용 여부
)

- 모델에 LoRA 어댑터를 추가하여 성능 향상 및 메모리 효율성을 높이는 과정을 실습합니다. 이를 통해 어댑터를 활용한 파인튜닝 방법을 익힙니다.


## 08-006 LLM JSON 파일 다운로드
- 이 실습에서는 JAEN 패키지를 사용하여 사전 정의된 LLM 구성 파일을 다운로드합니다. 이후 해당 JSON 파일을 사용하여 LLM 설정을 조정할 수 있습니다.


In [ ]:
from JAEN import download_file

download_file('llm') # llm.json

- JAEN 패키지를 통해 LLM 구성 파일을 다운로드하는 과정을 실습합니다. 다운로드한 JSON 파일은 모델 설정에 사용될 수 있습니다.


## 08-007 프롬프트 포매팅 함수 정의
- 이 실습에서는 주어진 예제의 지침(instruction)과 응답(output)을 기반으로 프롬프트를 포매팅하는 함수를 정의합니다. 이 함수를 사용하여 모델의 입력 형식을 통일할 수 있습니다.


In [ ]:
# 명령어 형식
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # eos 토큰 지정

def prompt_formatting(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []

    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN # eos을 완성된 문장 뒤에 추가
        texts.append(text)
    return {"text" : texts} # 사전 형태로 반환

- 프롬프트 포매팅 함수를 정의하여 명령어와 응답을 연결하는 과정을 실습합니다. 이를 통해 모델 입력을 일관되게 관리할 수 있습니다.


## 08-008 LLM JSON 데이터셋 로드 및 가공
- 이 실습에서는 JSON 형식의 LLM 데이터를 로드한 후, 정의한 `prompt_formatting` 함수를 사용하여 데이터셋을 가공합니다. 이를 통해 각 데이터 포인트를 모델의 입력 형식에 맞게 변환합니다.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="llm.json", split='train')

# 데이터셋 가공
# prompt_formatting을 데이터 적용
dataset = dataset.map(prompt_formatting, batched=True)

- LLM JSON 데이터를 로드하고 가공하여 모델 입력 형식에 맞게 변환하는 과정을 실습합니다. 이를 통해 데이터셋의 일관성을 유지할 수 있습니다.


## 08-009 파인튜닝을 위한 트레이너 설정
- 이 실습에서는 SFTTrainer를 사용하여 모델 파인튜닝을 위한 트레이너를 설정합니다. 다양한 훈련 매개변수를 설정하여 모델이 주어진 데이터셋에 적합하도록 학습할 수 있도록 합니다.


In [ ]:
from trl import SFTTrainer  # Supervised 파인튜닝 트레이너
from transformers import TrainingArguments  # 트랜스포머 모델 훈련을 위한 설정 관리
from unsloth import is_bfloat16_supported  # 시스템이 bfloat16(브레인 플로트) 형식을 지원하는지 확인하는 함수

# 파인튜닝을 위한 트레이너 설정
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,  # 학습 데이터
    dataset_text_field = "text", # 학습 데이터의 key (사전 key)
    max_seq_length = max_seq_length,  # 최대 토큰 수
    dataset_num_proc = 2,  # 데이터세트 전처리 프로세스 수
    packing = False,  # ConstantLengthDataset을 이용한 sequence 묶음 기능(학습 효율을 향상시키기 위한 방법)
    args = TrainingArguments(
        per_device_train_batch_size = 6,  # 장치에 전달할 small 배치 크기
        gradient_accumulation_steps = 4,  # grandient 누적 스텝
        warmup_steps = 5,  # warmup step 설정 (초기 학습률을 낮게 설정)
        num_train_epochs = 50,  # 학습 에폭
        learning_rate = 2e-4,  # 학습률 설"정
        fp16 = not is_bfloat16_supported(),  # FP16 사용 여부 (지원되지 않으면 False)
        bf16 = is_bfloat16_supported(),  # bfloat16 사용 여부 (지원되면 True)
        logging_steps = 1,  # 몇 스텝마다 로깅할지
        optim = "adamw_8bit",  # 최적화 방법 (8비트 AdamW 사용)
        weight_decay = 0.01,  # 가중치 감소 설정 (정칙화)
        lr_scheduler_type = "linear",  # 학습률 스케줄러 타입 (선형)
        seed = 1024,
        output_dir = output_dir,  # 훈련 결과 저장 디렉토리
        report_to = "none", # 로그 저장 연동
    )
)

- 모델 파인튜닝을 위한 트레이너 설정 과정을 실습합니다. 이를 통해 다양한 학습 파라미터를 설정하여 효과적인 모델 학습을 진행할 수 있습니다.


## 08-010 CUDA 장치 상태 및 메모리 획득
- 이 실습에서는 CUDA를 사용하여 GPU 장치의 상태와 메모리 사용량을 확인하는 방법을 보여줍니다. 이를 통해 모델 학습 시 GPU 자원 관리가 가능합니다.


In [ ]:
# CUDA 장치 상태 획득
gpu_stats = torch.cuda.get_device_properties(0)  # 첫 번째 GPU 장치의 속성 정보를 가져옴

# 사용 중인 메모리 획득
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)  # 사용 중인 GPU 메모리(GB 단위) 계산

# GPU 최대 메모리
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)  # GPU의 총 메모리 용량(GB 단위) 계산

# GPU 정보 및 메모리 상태 출력
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 용량 출력

- CUDA 장치 상태를 확인하고 GPU의 메모리 사용량을 측정하는 과정을 실습합니다. 이를 통해 GPU 자원의 효율적인 활용이 가능해집니다.


## 08-011 모델 파인튜닝 수행
- 이 실습에서는 설정한 트레이너를 사용하여 모델 파인튜닝을 수행합니다. 모델이 주어진 데이터셋을 기반으로 학습하게 됩니다.


In [ ]:
import os

# 파인튜닝 수행
trainer_stats = trainer.train()

- 모델 파인튜닝을 수행하여 학습 결과를 얻는 과정을 실습합니다. 이를 통해 모델의 성능을 개선할 수 있습니다.


## 08-012 학습 후 추론 결과 확인
- 이 실습에서는 설정한 모델을 사용하여 입력 쿼리에 대한 추론을 수행합니다. 모델이 주어진 입력에 대해 적절한 응답을 생성하는 과정을 확인할 수 있습니다.


In [ ]:
FastLanguageModel.for_inference(model)  # 추론 모드 설정

# 입력 query
query = prompt.format("삼성전자 캠퍼스에 대해서 알려주세요.",  "")

# 입력 데이터 토큰화
input = tokenizer(query, return_tensors="pt").to('cuda') # GPU 필수

# 추론
# gpu로 약
output = model.generate(**input, max_new_tokens=512, use_cache=True, repetition_penalty=2.0)

# 출력 토큰을 문자으로 변환
print(tokenizer.decode(output[0], skip_special_tokens=True))

- 입력 쿼리를 기반으로 모델이 생성한 출력을 확인하는 과정을 실습합니다. 이를 통해 모델의 응답 품질을 평가할 수 있습니다.


## 08-013 학습된 모델 저장
- 이 실습에서는 학습된 모델을 지정된 양자화 방법을 사용하여 GGUF 포맷으로 저장하는 방법을 보여줍니다. 저장된 모델은 추후 불러와 사용할 수 있습니다.


In [ ]:
# 학습 모델 저장
quantization_methods = ["f16", "q8_0", "q4_k_m"]  # 사용할 양자화 방법들을 정의

# GGUF 포맷으로 양자화된 모델을 저장
model.save_pretrained_gguf(
    output_dir,
    tokenizer,  # 토크나이저 정보도 함께 저장
    quantization_method=quantization_methods[2]  # 양자화 방법 지정
)

# GGUF(Grok GPT Unfiltered Format)은 AI 언어 모델의 압축 및 최적화된 포맷 중 하나

- 학습된 모델을 다양한 양자화 방법으로 저장하여 모델의 활용도를 높이는 과정을 실습합니다.
